In [ ]:
#!pip install tensorflow

In [19]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ReduceLROnPlateau

import shutil
import os
from sklearn.model_selection import train_test_split

In [12]:
# Parameter
IMG_SIZE = 224 
BATCH_SIZE = 32
EPOCHS = 100

In [3]:
# Direktori dataset
dataset_dir = "dataset_batik"
train_dir = "train_batik"
test_dir = "test_batik"
val_dir = "val_batik"  # Menambahkan direktori untuk validasi

# Membuat folder untuk train, test, dan val jika belum ada
os.makedirs(train_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)
os.makedirs(val_dir, exist_ok=True)

# Daftar semua subfolder (kelas)
subfolders = [f for f in os.listdir(dataset_dir) if os.path.isdir(os.path.join(dataset_dir, f))]

# Membagi data menjadi train, test, dan val
for subfolder in subfolders:
    subfolder_path = os.path.join(dataset_dir, subfolder)
    
    # Membuat folder untuk kelas di dalam folder train, test, dan val
    train_subfolder = os.path.join(train_dir, subfolder)
    test_subfolder = os.path.join(test_dir, subfolder)
    val_subfolder = os.path.join(val_dir, subfolder)
    
    os.makedirs(train_subfolder, exist_ok=True)
    os.makedirs(test_subfolder, exist_ok=True)
    os.makedirs(val_subfolder, exist_ok=True)
    
    # Ambil semua gambar di dalam subfolder
    images = [f for f in os.listdir(subfolder_path) if os.path.isfile(os.path.join(subfolder_path, f))]
    
    # Bagi data menggunakan train_test_split dua tahap
    train_val_images, test_images = train_test_split(images, test_size=0.1, random_state=42)  # 10% untuk test
    train_images, val_images = train_test_split(train_val_images, test_size=0.125, random_state=42)  # 0.125 * 0.9 = 0.1 untuk val
    
    # Pindahkan gambar ke folder train, val, dan test
    for image in train_images:
        old_image_path = os.path.join(subfolder_path, image)
        new_image_path = os.path.join(train_subfolder, image)
        shutil.copy(old_image_path, new_image_path)

    for image in val_images:
        old_image_path = os.path.join(subfolder_path, image)
        new_image_path = os.path.join(val_subfolder, image)
        shutil.copy(old_image_path, new_image_path)

    for image in test_images:
        old_image_path = os.path.join(subfolder_path, image)
        new_image_path = os.path.join(test_subfolder, image)
        shutil.copy(old_image_path, new_image_path)

print("Data telah dibagi menjadi folder train, val, dan test dengan komposisi 80%, 10%, dan 10%.")


Data telah dibagi menjadi folder train, val, dan test dengan komposisi 80%, 10%, dan 10%.


In [5]:
# Augmentasi Data
train_datagen = ImageDataGenerator(
    rescale=1.0/255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode="nearest"
)
test_datagen = ImageDataGenerator(rescale=1.0/255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(IMG_SIZE, IMG_SIZE),  # Ubah ini menjadi tuple
    batch_size=BATCH_SIZE,
    class_mode="categorical"
)

val_generator = test_datagen.flow_from_directory(
    val_dir,
    target_size=(IMG_SIZE, IMG_SIZE),  # Ubah ini menjadi tuple
    batch_size=BATCH_SIZE,
    class_mode="categorical"
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(IMG_SIZE, IMG_SIZE),  # Ubah ini menjadi tuple
    batch_size=BATCH_SIZE,
    class_mode="categorical"
)

# Informasi jumlah kelas
num_classes = len(train_generator.class_indices)

Found 2128 images belonging to 15 classes.
Found 313 images belonging to 15 classes.
Found 276 images belonging to 15 classes.


In [6]:
# Verifikasi jumlah gambar di train dan test
for subfolder in os.listdir(train_dir):
    subfolder_path = os.path.join(train_dir, subfolder)
    if os.path.isdir(subfolder_path):
        images = os.listdir(subfolder_path)
        print(f"Train class '{subfolder}' has {len(images)} images.")

for subfolder in os.listdir(test_dir):
    subfolder_path = os.path.join(test_dir, subfolder)
    if os.path.isdir(subfolder_path):
        images = os.listdir(subfolder_path)
        print(f"Test class '{subfolder}' has {len(images)} images.")


for subfolder in os.listdir(val_dir):
    subfolder_path = os.path.join(val_dir, subfolder)
    if os.path.isdir(subfolder_path):
        images = os.listdir(subfolder_path)
        print(f"Val class '{subfolder}' has {len(images)} images.")


Train class 'batik_celup' has 143 images.
Train class 'batik_cendrawasih' has 141 images.
Train class 'batik_dayak' has 156 images.
Train class 'batik_geblek_renteng' has 163 images.
Train class 'batik_insang' has 153 images.
Train class 'batik_kawung' has 162 images.
Train class 'batik_lasem' has 147 images.
Train class 'batik_megamendung' has 168 images.
Train class 'batik_parang' has 164 images.
Train class 'batik_poleng' has 154 images.
Train class 'batik_pring' has 31 images.
Train class 'batik_sekar' has 117 images.
Train class 'batik_sidoluhur' has 123 images.
Train class 'batik_tambal' has 145 images.
Train class 'batik_truntum' has 161 images.
Test class 'batik_celup' has 19 images.
Test class 'batik_cendrawasih' has 18 images.
Test class 'batik_dayak' has 20 images.
Test class 'batik_geblek_renteng' has 21 images.
Test class 'batik_insang' has 20 images.
Test class 'batik_kawung' has 21 images.
Test class 'batik_lasem' has 19 images.
Test class 'batik_megamendung' has 22 imag

In [14]:
input_shape = (IMG_SIZE, IMG_SIZE, 3)

# Membangun model CNN
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
    layers.MaxPooling2D((2, 2)),
    
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(num_classes, activation='softmax')
])

# Menampilkan summary model
model.summary()


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_10 (Conv2D)              │ (None, 222, 222, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_10 (MaxPooling2D) │ (None, 111, 111, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_11 (Conv2D)              │ (None, 109, 109, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_11 (MaxPooling2D) │ (None, 54, 54, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_12 (Conv2D)              │ (None, 52, 52, 64)     │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_12 (MaxPooling2D) │ (None, 26, 26, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_13 (Conv2D)              │ (None, 24, 24, 64)     │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_13 (MaxPooling2D) │ (None, 12, 12, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_14 (Conv2D)              │ (None, 10, 10, 64)     │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_14 (MaxPooling2D) │ (None, 5, 5, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_2 (Flatten)             │ (None, 1600)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 64)             │       102,464 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 15)             │           975 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 233,615 (912.56 KB)

 Trainable params: 233,615 (912.56 KB)

 Non-trainable params: 0 (0.00 B)

In [15]:
# Kompilasi model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

history = model.fit(
    train_generator,
    epochs= EPOCHS,
    batch_size=BATCH_SIZE,
    verbose=1,
    validation_data = val_generator
)

Epoch 1/100
67/67 ━━━━━━━━━━━━━━━━━━━━ 167s 2s/step - accuracy: 0.0764 - loss: 2.6922 - val_accuracy: 0.0895 - val_loss: 2.6566
Epoch 2/100
67/67 ━━━━━━━━━━━━━━━━━━━━ 158s 2s/step - accuracy: 0.1042 - loss: 2.6250 - val_accuracy: 0.1757 - val_loss: 2.4383
Epoch 3/100
67/67 ━━━━━━━━━━━━━━━━━━━━ 160s 2s/step - accuracy: 0.1676 - loss: 2.4652 - val_accuracy: 0.1374 - val_loss: 2.4063
Epoch 4/100
67/67 ━━━━━━━━━━━━━━━━━━━━ 172s 2s/step - accuracy: 0.2135 - loss: 2.3124 - val_accuracy: 0.2364 - val_loss: 2.2094
Epoch 5/100
67/67 ━━━━━━━━━━━━━━━━━━━━ 159s 2s/step - accuracy: 0.2548 - loss: 2.2318 - val_accuracy: 0.2939 - val_loss: 2.1080
Epoch 6/100
67/67 ━━━━━━━━━━━━━━━━━━━━ 174s 2s/step - accuracy: 0.2700 - loss: 2.1802 - val_accuracy: 0.3099 - val_loss: 2.0191
Epoch 7/100
67/67 ━━━━━━━━━━━━━━━━━━━━ 171s 2s/step - accuracy: 0.3087 - loss: 2.0963 - val_accuracy: 0.3450 - val_loss: 1.9674
Epoch 8/100
67/67 ━━━━━━━━━━━━━━━━━━━━ 181s 2s/step - accuracy: 0.3420 - loss: 1.9988 - val_accuracy: 0.

In [16]:
model.save("model_100_epochs_scartch.h5")

In [ ]:
model = tf.keras.models.load_model('model_100_epochs_scartch.h5')

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Callback
reduce_lr = ReduceLROnPlateau(monitor="val_loss", factor=0.2, patience=3, min_lr=1e-6)

history = model.fit(
    train_generator,
    epochs= 200,
    initial_epoch=100,
    batch_size=BATCH_SIZE,
    verbose=1,
    validation_data = val_generator
    callbacks=[reduce_lr]
)

Epoch 101/200
67/67 ━━━━━━━━━━━━━━━━━━━━ 113s 1s/step - accuracy: 0.8724 - loss: 0.3684 - val_accuracy: 0.7604 - val_loss: 1.0514
Epoch 102/200
67/67 ━━━━━━━━━━━━━━━━━━━━ 94s 1s/step - accuracy: 0.9136 - loss: 0.2573 - val_accuracy: 0.7636 - val_loss: 1.0260
Epoch 103/200
67/67 ━━━━━━━━━━━━━━━━━━━━ 95s 1s/step - accuracy: 0.8826 - loss: 0.3368 - val_accuracy: 0.7732 - val_loss: 1.0151
Epoch 104/200
67/67 ━━━━━━━━━━━━━━━━━━━━ 95s 1s/step - accuracy: 0.9304 - loss: 0.2241 - val_accuracy: 0.7093 - val_loss: 1.1407
Epoch 105/200
67/67 ━━━━━━━━━━━━━━━━━━━━ 107s 1s/step - accuracy: 0.9153 - loss: 0.2560 - val_accuracy: 0.7444 - val_loss: 1.0019
Epoch 106/200
67/67 ━━━━━━━━━━━━━━━━━━━━ 96s 1s/step - accuracy: 0.8969 - loss: 0.3171 - val_accuracy: 0.7284 - val_loss: 1.1253
Epoch 107/200
67/67 ━━━━━━━━━━━━━━━━━━━━ 102s 1s/step - accuracy: 0.8883 - loss: 0.3521 - val_accuracy: 0.7316 - val_loss: 1.1885
Epoch 108/200
67/67 ━━━━━━━━━━━━━━━━━━━━ 99s 1s/step - accuracy: 0.8980 - loss: 0.3184 - val_a

KeyboardInterrupt: 